In [62]:
#Путь к файлу
pathToFile = 'E:\\загрузки'
fileName = 'predict.csv'

In [63]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor

In [64]:
#Чтение данных в формате csv
os.chdir(pathToFile)
data = pd.read_csv(fileName, delimiter = ',', header=0)

In [65]:
#Первичная обработка данных

#Проверка на пустые столбцы
notNullColumns = [col for col in data.columns if data.loc[:, col].notna().any()]

#Оставляем только столбцы с данными
data = data[notNullColumns]

#Убираем пустые значения в строках
data = data.dropna()

#Получаем корреляционную матрицу для проверки избыточности информации
corrMatix = data.drop(['c5'], axis=1).corr()

#Так как матрица симметрична оставляем только половину для удобства работы
j=0
for col in corrMatix.columns:
    corrMatix[col][j:] = 0
    j += 1

#Получаем имена столбцов, корреляция которых выше 0,9
corrColumns = [col for col in corrMatix.columns if len(corrMatix.loc[corrMatix[col] > 0.9, col]) > 0]

#Удаляем зависимые столбцы
data = data.drop(corrColumns, axis=1)

In [66]:
#Разделение выборки на предикторы (predictors) и отклик (response)
resp = data['c5']
pred = data.drop(['c5'], axis=1)

#Определяем число для разбиение выборки 30 на 70
num = int(len(data)*0.3)

#Разделение выборки на тестовую (test) и обучающую (training)
predTest,predTrn = pred[:num], pred[num:]
respTest,respTrn = resp[:num], resp[num:]

In [67]:
#Переводим данные в матричный вид
d_train = xgb.DMatrix(predTrn, label=respTrn)
d_test = xgb.DMatrix(predTest, label=respTest)

In [68]:
#Подбираем параметры модели
xgb_params = {"max_depth": 10, "eta": 0.5}

In [69]:
#Обучение модели
model = xgb.train(params=xgb_params, dtrain=d_train, num_boost_round=10)

In [70]:
#Обучаем gradient boosting model на тренировочных данных
predictions = model.predict(d_test)

In [71]:
#Вычисляем значение R^2
r2 = r2_score(respTest,predictions)
print('R^2: ', round(r2, 3))

#Вычисляем mean squared error
mse = mean_squared_error(respTest,predictions)
print('MSE: ', round(mse, 2))

R^2:  0.601
MSE:  25.13
